In [1]:
import os
import pickle
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import time

2022-07-15 14:42:32.811868: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
print(os.getcwd())
os.chdir("/home/km/temp/DeepDEP/")
print(os.getcwd())

/home/km/temp/DeepDEP/prediction/code
/home/km/temp/DeepDEP


In [3]:
def load_data(filename):
    data = []
    gene_names = []
    data_labels = []
    lines = open(filename).readlines()
    sample_names = lines[0].replace('\n', '').split('\t')[1:]
    dx = 1

    for line in lines[dx:]:
        values = line.replace('\n', '').split('\t')
        gene = str.upper(values[0])
        gene_names.append(gene)
        data.append(values[1:])
    data = np.array(data, dtype='float32')
    data = np.transpose(data)

    return data, data_labels, sample_names, gene_names

def AE_dense_3layers(input_dim, first_layer_dim, second_layer_dim, third_layer_dim, activation_func, init='he_uniform'):
    print('input_dim = ', input_dim)
    print('first_layer_dim = ', first_layer_dim)
    print('second_layer_dim = ', second_layer_dim)
    print('third_layer_dim = ', third_layer_dim)
    print('init = ', init)
    
    with tf.device('/cpu:0'):
        model = Sequential()
        model.add(Dense(first_layer_dim, input_dim = input_dim, activation = activation_func, kernel_initializer = init))
        model.add(Dense(second_layer_dim, input_dim = first_layer_dim, activation = activation_func, kernel_initializer = init))
        model.add(Dense(third_layer_dim, input_dim = second_layer_dim, activation = activation_func, kernel_initializer = init))
        model.add(Dense(second_layer_dim, input_dim = third_layer_dim, activation = activation_func, kernel_initializer = init))
        model.add(Dense(first_layer_dim, input_dim = second_layer_dim, activation = activation_func, kernel_initializer = init))
        model.add(Dense(input_dim, input_dim = first_layer_dim, activation = activation_func, kernel_initializer = init))
    
    return model

def save_weight_to_pickle(model, file_name):
    print('saving weights')
    weight_list = []
    for layer in model.layers:
        weight_list.append(layer.get_weights())
    with open(file_name, 'wb') as handle:
        pickle.dump(weight_list, handle)

In [4]:
TCGA_PATH = "preprocessing/TCGA_INDEX/CUSTOM/"
SAVE_PATH = "prediction/custom_model/"

In [5]:
# load TCGA mutation data, substitute here with other genomics
data_tcga, data_labels_tcga, sample_names_tcga, property_names_tcga = load_data(TCGA_PATH + "tcga_cna_data_paired_with_ccl_custom.txt")
print("\n\nDatasets successfully loaded.")



Datasets successfully loaded.


In [6]:
data_tcga.shape

(8709, 5915)

In [34]:
input_dim = data_tcga.shape[1]
first_layer_dim = 1000
second_layer_dim = 100
third_layer_dim = 50
batch_size = 64
epoch_size = 100
activation_function = 'relu'
init = 'he_uniform'

In [35]:
model_save_name = "premodel_tcga_custom_cna_%d_%d_%d" % (first_layer_dim, second_layer_dim, third_layer_dim)

In [36]:
t = time.time()
model = AE_dense_3layers(input_dim = input_dim, first_layer_dim = first_layer_dim, 
                         second_layer_dim=second_layer_dim, third_layer_dim=third_layer_dim, 
                         activation_func=activation_function, init=init)
# tf.keras.utils.plot_model(model, show_shapes=True,  show_dtype=True, to_file='model.png')

input_dim =  5915
first_layer_dim =  1000
second_layer_dim =  100
third_layer_dim =  50
init =  he_uniform


In [37]:
with tf.device('/cpu:0'):
    model.compile(loss = 'mse', optimizer = 'adam')
    model.fit(data_tcga, data_tcga, epochs=epoch_size, 
              batch_size=batch_size, shuffle=True, verbose = 1)

Epoch 1/100
137/137 [==============================] - 7s 50ms/step - loss: 0.0718
Epoch 2/100
137/137 [==============================] - 7s 49ms/step - loss: 0.0453
Epoch 3/100
137/137 [==============================] - 7s 48ms/step - loss: 0.0385
Epoch 4/100
137/137 [==============================] - 7s 49ms/step - loss: 0.0347
Epoch 5/100
137/137 [==============================] - 7s 49ms/step - loss: 0.0322
Epoch 6/100
137/137 [==============================] - 7s 49ms/step - loss: 0.0316
Epoch 7/100
137/137 [==============================] - 7s 49ms/step - loss: 0.0303
Epoch 8/100
137/137 [==============================] - 7s 49ms/step - loss: 0.0305
Epoch 9/100
137/137 [==============================] - 7s 49ms/step - loss: 0.0294
Epoch 10/100
137/137 [==============================] - 7s 49ms/step - loss: 0.0289
Epoch 11/100
137/137 [==============================] - 7s 49ms/step - loss: 0.0281
Epoch 12/100
137/137 [==============================] - 7s 49ms/step - loss: 0.0275
E

In [38]:
cost = model.evaluate(data_tcga, data_tcga, verbose = 0)
print('\n\nAutoencoder training completed in %.1f mins.\n with testloss:%.4f' % ((time.time()-t)/60, cost))



Autoencoder training completed in 11.3 mins.
 with testloss:0.0192


In [39]:
save_weight_to_pickle(model,SAVE_PATH + model_save_name + '.pickle')
print("\nResults saved in %s%s.pickle\n\n" % (SAVE_PATH, model_save_name))

saving weights

Results saved in prediction/custom_model/premodel_tcga_custom_cna_1000_100_50.pickle


